# Project 1: Clustering with SOM
## Imports and Setup

Basic imports, sklearn utilities, MiniSom, and matplotlib configuration. Includes robust MiniSom import/installation.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from minisom import MiniSom

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## Data Loading and Preparation

Load the digits dataset, scale the features, and split into training, validation, and test sets (60/20/20 split).

In [3]:
# Load the digits dataset
digits = load_digits()
X = digits.data # Original data (8x8=64 features)
y = digits.target # Labels (0-9)
print(f"Digits dataset loaded: {X.shape[0]} samples, {X.shape[1]} features.")
print(f"Number of classes: {len(np.unique(y))}")

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print(f"Data scaled using {type(scaler).__name__}.")

# 1. Split into Initial Train and Final Test sets
X_train_init, X_test, y_train_init, y_test, X_train_init_orig, X_test_orig = train_test_split(
    X_scaled, y, X, # Split scaled X, y, AND original X
    test_size=0.2,    # 80% initial train, 20% final test
    random_state=42,
    stratify=y )
print(f"Initial data split: Initial Train ({X_train_init.shape[0]}), Final Test ({X_test.shape[0]})")

# 2. Split Initial Train into Final Train and Validation sets
X_train_final, X_val, y_train_final, y_val, X_train_final_orig, X_val_orig = train_test_split(
    X_train_init, y_train_init, X_train_init_orig, # Split initial train data (scaled and original)
    test_size=0.25,
    random_state=42,
    stratify=y_train_init
)
print(f"Train/Validation split: Final Train ({X_train_final.shape[0]}), Validation ({X_val.shape[0]})")
print("Digits data prepared and split into Train/Validation/Test sets.")
print("-" * 30)

Digits dataset loaded: 1797 samples, 64 features.
Number of classes: 10
Data scaled using MinMaxScaler.
Initial data split: Initial Train (1437), Final Test (360)
Train/Validation split: Final Train (1077), Validation (360)
Digits data prepared and split into Train/Validation/Test sets.
------------------------------


## SOM Implementation & Training

Define SOM parameters and train two SOMs (4x4 and 20x20) on the final training data. Quantization Error on train and validation sets is calculated. Hyperparameters should be tuned based on validation performance (QE and classification accuracy from Task 2 validation).

In [4]:
# Define SOM parameters
som_grid_size_small = (4, 4)
som_grid_size_large = (20, 20)
n_features = X_train_final.shape[1] 

# Hyperparameters for Small SOM
sigma_small = 1.0
lr_small = 0.5              
iterations_small = 10000
init_method_small = "PCA"

# Hyperparameters for Large SOM
sigma_large = 3.0
lr_large = 0.5             
iterations_large = 15000
init_method_large = "Random"

print("SOM Hyperparameters defined:")
print(f"  Small (4x4): Sigma={sigma_small}, LR={lr_small}, Iter={iterations_small}, Init={init_method_small}")
print(f"  Large (20x20): Sigma={sigma_large}, LR={lr_large}, Iter={iterations_large}, Init={init_method_large}")
print("-" * 30)

SOM Hyperparameters defined:
  Small (4x4): Sigma=1.0, LR=0.5, Iter=10000, Init=PCA
  Large (20x20): Sigma=3.0, LR=0.5, Iter=15000, Init=Random
------------------------------
